#Running memristor model using weights and input of HAR model using pytorch and vivado
in this program i have used human activity recognition(HAR) dataset, in previously i have used same dataset to find the important feature from the same dataset where i have used SHAP XAI technique where i have by increased and decred the feature and got different accuracy so have used 95 feature where accuracy is near to by using all 561 feature.

i have used this reference paper "An FPGA-based memristor emulator for artificial neural network" where they used a MNIST dataset,
step 1: used MNIST dataset trained and tested using single layer perceptron(SLP).
step 2: used the weights of SLP and quantized between (0-2) with step size = 0.25 with 8-valued memristor. then converted to fixed point number which can be used in verilog when running descrete memristor model.
step 3: the features are scaled between (-4, 4) then converted to fixed point number where input and weights can be expressed in 4 digits for integer, 2 digits for decimal and 1 digit for sign.


i have replicated the same paper i have 1st used the same dataset and same procedure later chnaged the dataset to HAR used 95 feature instead 561 feature. then trained and tested using single layer perceptron (SLP) saved the best model and their weights later followed above step 1, 2 and 3. converted to .txt file so that can be uploaded while writing a program for memrristor model.  

memristor model: as mentioned in the paper if consider MNIST dataset there are 784 inputs (28x28 image pixel) and there are 10 handwritten digit so 784*10=7840 weights and each pixel has one memristor so there are 7840 memristor module.

mow in this i have used HAR dataset in that 95 feature so there are 95 inputs and there are 6 class (0-5) so 95*6=570 memristor module.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np

# Load and split HAR dataset (features + class)
X_train_full = pd.read_csv("/content/X_train_reduced_95(1).csv", header=0)
X_test_full  = pd.read_csv("/content/X_test_reduced_95(1).csv", header=0)

X_train = X_train_full.iloc[:, :-1]
y_train = X_train_full.iloc[:, -1].astype(int) - 1

X_test = X_test_full.iloc[:, :-1]
y_test  = X_test_full.iloc[:, -1].astype(int) - 1

#  Scale inputs to (-4, 4)
X_min = X_train.min()
X_max = X_train.max()

X_train_scaled = ((X_train - X_min) / (X_max - X_min)) * 8 - 4
X_test_scaled  = ((X_test  - X_min) / (X_max - X_min)) * 8 - 4

#  Convert to tensors for PyTorch training
X_train_t = torch.tensor(X_train_scaled.values, dtype=torch.float32)
y_train_t = torch.tensor(y_train.values, dtype=torch.long)

X_test_t = torch.tensor(X_test_scaled.values, dtype=torch.float32)
y_test_t = torch.tensor(y_test.values, dtype=torch.long)

train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=64, shuffle=True)
test_loader  = DataLoader(TensorDataset(X_test_t, y_test_t), batch_size=128, shuffle=False)

#  Define Single-Layer Perceptron (SLP)
class SLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(SLP, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

# Train the SLP
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SLP(input_dim=X_train.shape[1], num_classes=6).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(10):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for X, y in train_loader:
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()

    acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/10] | Loss: {total_loss/len(train_loader):.4f} | Accuracy: {acc:.2f}%")

# Evaluate model on test set
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for X, y in test_loader:
        X, y = X.to(device), y.to(device)
        outputs = model(X)
        _, predicted = torch.max(outputs, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()

print(f"✅ Test Accuracy: {100 * correct / total:.2f}%")

# Scale weights (0 → 2) and quantize to Q4.2
weights = model.fc.weight.data.cpu().numpy()

# Scale weights to [0, 2]
# Scale weights to [0, 2]
w_min, w_max = weights.min(), weights.max()
weights_scaled = ((weights - w_min) / (w_max - w_min)) * 2

# Quantize to Q4.2 (×4)
weights_fixed = (weights_scaled * 4).round().astype(int).clip(0, 8)

# Save weights
pd.DataFrame(weights_fixed).to_csv("weights_HAR_fixed.csv", index=False, header=False)

# Quantize and save scaled inputs (X_train, X_test)
def quantize_and_save(X_scaled, y, name):
    X_fixed = (X_scaled * 4).round().astype(int).clip(-16, 16)
    df_fixed = pd.concat([pd.DataFrame(X_fixed), pd.Series(y.values)], axis=1)
    df_fixed.to_csv(f"{name}_fixed.csv", index=False, header=False)
    print(f"✅ Saved: {name}_fixed.csv")

quantize_and_save(X_train_scaled, y_train, "HAR_train")
quantize_and_save(X_test_scaled,  y_test,  "HAR_test")

print("🎉 All fixed-point datasets and weights saved successfully.")


In [ ]:
# Save raw, scaled, and fixed-point weights
# 7.1 Get raw learned weights
weights = model.fc.weight.data.cpu().numpy()  # shape: [num_classes, num_features]
pd.DataFrame(weights).to_csv("weights_HAR_raw.csv", index=False, header=False)
print("Saved: weights_HAR_raw.csv (original learned weights)")

# 7.2 Scale weights to range (0 → 2)
w_min, w_max = weights.min(), weights.max()
weights_scaled = ((weights - w_min) / (w_max - w_min)) * 2
pd.DataFrame(weights_scaled).to_csv("weights_HAR_scaled_0to2.csv", index=False, header=False)
print(" Saved: weights_HAR_scaled_0to2.csv (scaled to 0–2)")

# 7.3 Quantize to Q4.2 fixed-point (×4)
weights_fixed = (weights_scaled * 4).round().astype(int).clip(0, 8)
pd.DataFrame(weights_fixed).to_csv("weights_HAR_fixed.csv", index=False, header=False)
print(" Saved: weights_HAR_fixed.csv (Q4.2 integers, 0–8)")

# 7.4 Optional: Save as one-value-per-line .txt file (for Verilog)
weights_fixed_flat = weights_fixed.flatten()
np.savetxt("weights_HAR_fixed1.txt", weights_fixed_flat, fmt="%d")
print(" Saved: weights_HAR_fixed1.txt (one integer per line, ready for Verilog)")

this code snippet for extraing single row feature for testing in vivado.

In [ ]:
import pandas as pd
import numpy as np

# Load fixed-point test set (features + label)
df = pd.read_csv("/content/HAR_test_fixed.csv", header=None)

row_index = 1316   # Python is 0-based → row 3
sample = df.iloc[row_index, :-1]   # exclude label column

np.savetxt("X_test_row1316.txt", sample.values, fmt="%d")
print(" Saved 3rd test sample to X_test_row1316.txt")